# Second-Cut Run 68 $\gamma$ and Neutron Direct Backgrounds

My notebook N-MISC-18-003 shows (pgs 67) simulations that were started to assess the direct gamma and neutron backgrounds for a Pu/Be source running in the K100 Run 68 setup. Specifically testing out the .h5 files created using the scripts mentioned on pg 52 of the above notebook reference. 

In [1]:
#play around with some hits data stored in h5 file
#===============to suppress h5py warning see:
#https://github.com/h5py/h5py/issues/961
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()
f = h5py.File("/data/chocula/villaa/k100Sim_Data/captureCalhdf5/Run68_gdirect_bknd_R68_PuBe_0x0006_1551197784.h5","r")

for i in f:
    print(i)

hits = f['geant4/hits']

geant4


In [2]:
import numpy as np
print(np.shape(hits))
print(hits[0,:])

(21687, 22)
[ 3.27000000e+02  1.00100000e+03  1.00062000e+05  0.00000000e+00
  2.11200000e+03  2.13291287e-08  0.00000000e+00  7.11570734e-04
 -6.29334487e-03  1.27301707e-03 -4.21449958e+02  1.20427548e+00
 -2.87385739e+02  8.80377810e+05  7.44766995e-04 -4.41958150e-03
  4.47133564e-03 -4.31153289e+02  5.87855884e+01 -3.45641340e+02
  8.39545241e+05  0.00000000e+00]


In [3]:
#try to label events with consecutive and unique labels
ev = hits[:,0]

diffs = np.append(np.diff(ev),1)
diff_divide = np.copy(diffs)
diff_divide[diff_divide==0] = 1 #replace some elements with unity
diffs = diffs/diff_divide
#print(diffs[0:300])
#print(diff_divide[0:300])

newev = np.cumsum(diffs)
print(newev[0:300])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  2.  2.  2.  2.  2.  2.  3.  3.  3.  3.  3.
  3.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  4.  4.  5.  5.  5.  6.  6.  6.  6.  6.  6.  7.  7.  8.  8.  8.  8.
  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.
  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  9.  9.  9.  9.  9.  9.  9.  9.
  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.
  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9. 10. 10. 10. 10.
 10. 10. 11. 11. 11. 11. 11. 12. 12. 12. 12. 12. 12. 13. 13. 13. 13. 13.
 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13.
 13. 13. 13. 13. 13. 14. 14. 14. 14. 15. 15. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16

In [4]:
#select the poly block
cPoly = np.zeros(np.shape(hits)[0],dtype=bool)


cPoly[(hits[:,1]==1000)] = True
print(np.sum(cPoly))
print(np.shape(cPoly))
print(np.shape(hits))

0
(21687,)
(21687, 22)


In [5]:
#reminder of file structure
#EV  DT  TS  P Type  E1  D3  PX3 PY3 PZ3 X3  Y3  Z3  time3 PX1 PY1 PZ1 X1  Y1  Z1  time1 nCap
#python index below
#0    1   2 3   4     5   6   7   8   9  10  11  12    13   14  15  16 17  18  19   20    21

In [6]:
#I want to find the edges in Y because those are the places the poly surfaces that I'm interested in are.
#I can do that by a histogram

#first some hit-level cuts
cHVDet = np.zeros(np.shape(hits)[0],dtype=bool)
cZeroEdep = np.zeros(np.shape(hits)[0],dtype=bool)
cNeutron = np.zeros(np.shape(hits)[0],dtype=bool)
cGamma = np.zeros(np.shape(hits)[0],dtype=bool)
cPY3neg = np.zeros(np.shape(hits)[0],dtype=bool)
cWestYPolyEdge = np.zeros(np.shape(hits)[0],dtype=bool)

cHVDet[hits[:,1]==1] = True
cZeroEdep[hits[:,6]==0] = True
cNeutron[hits[:,4]==2112] = True
cGamma[hits[:,4]==22] = True
cPY3neg[hits[:,8]<0] = True #going in y direction away from barrel source
cWestYPolyEdge[hits[:,11]==330.2] = True #right at the West poly edge closest to cryostat

#now some event-level cuts
evWithHVhits = newev[cHVDet & ~cZeroEdep]
cWithHVHits = np.isin(newev,evWithHVhits)
print(np.sum(cWithHVHits))
print(np.shape(np.unique(evWithHVhits)))

finalypos_n = hits[:,11]
finalypos_n = finalypos_n[cZeroEdep & cNeutron & cPY3neg & cPoly]/1000 # get things in meters instead of mm
finalypos_g = hits[:,11]
finalypos_g = finalypos_g[cZeroEdep & cGamma & cPY3neg & cPoly]/1000 # get things in meters instead of mm
print(np.shape(finalypos_g))
print(finalypos_g[0:100])

284
(20,)
(0,)
[]


In [7]:
import pandas as pd

NRhits = hits[:,[0,4,6]]
NRhits = NRhits[~cZeroEdep & cHVDet]

In [21]:
print(pd.DataFrame(data=NRhits))
print(pd.DataFrame(data=NRhits).groupby([0,1],axis=0)[2].apply(list))
print(pd.DataFrame(data=NRhits).groupby([0,1],axis=0)[2].apply(list).index.values)
print(np.max(pd.DataFrame(data=NRhits).groupby([0,1],axis=0).size()))

           0     1         2
0     2052.0  11.0  0.073041
1    24416.0  11.0  0.735581
2    24416.0  11.0  0.586006
3    24416.0  11.0  0.039483
4   149605.0  11.0  0.099736
5   331506.0  11.0  0.044865
6   331506.0  11.0  0.018475
7   331506.0  11.0  0.326680
8   396545.0  11.0  0.038561
9   515578.0  11.0  0.012160
10  515578.0  11.0  0.229061
11  664291.0  11.0  0.000160
12  664291.0  11.0  0.164225
13  695725.0  11.0  0.130093
14  695725.0  11.0  0.031124
15  716253.0  22.0  0.001844
16  716253.0  11.0  0.089064
17  716253.0  11.0  0.016189
18  716253.0  11.0  0.005480
19  716253.0  11.0  0.075444
20  716253.0  11.0  0.002547
21  757682.0  11.0  0.027608
22  826463.0  11.0  0.027007
23  894315.0  11.0  0.001060
24  894315.0  11.0  0.030582
25  908285.0  11.0  0.019847
0         1   
2052.0    11.0                                      [0.07304051082]
24416.0   11.0          [0.7355806226, 0.5860056757, 0.03948326892]
149605.0  11.0                                      [0.09973560942

In [11]:


for i in pd.DataFrame(data=NRhits).groupby([0,1],axis=0)[2].apply(list):
    print(i)
    print(np.shape(i))
    

[0.07304051082]
(1,)
[0.7355806226, 0.5860056757, 0.03948326892]
(3,)
[0.09973560942]
(1,)
[0.04486459351, 0.01847534293, 0.3266799633]
(3,)
[0.03856060989]
(1,)
[0.01216023477, 0.229061086]
(2,)
[0.0001595338792, 0.164225234]
(2,)
[0.1300926768, 0.03112367635]
(2,)
[0.08906361545, 0.01618901408, 0.005480488057, 0.07544407764, 0.002546628628]
(5,)
[0.001844]
(1,)
[0.027607886]
(1,)
[0.0270067064]
(1,)
[0.001060361975, 0.03058239984]
(2,)
[0.01984719452]
(1,)
